## 빅데이터 분석기사 
---
### Census Income 
- 사람들의 나이, 교육 수준, 결혼 여부 등의 특성과 연 소득이 5만 달러를 넘는지에 대한 정보

### 테스트로 내가 알아서 해보기!

In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [2]:
X_test = pd.read_csv('/Users/hayeon/Downloads/census_X_test.csv')
X_train = pd.read_csv('/Users/hayeon/Downloads/census_X_train.csv')
y_train = pd.read_csv('/Users/hayeon/Downloads/census_y_train.csv')

In [3]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30162 entries, 0 to 30161
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   age             30162 non-null  int64 
 1   workclass       30162 non-null  object
 2   education_num   30162 non-null  int64 
 3   marital_status  30162 non-null  object
 4   occupation      30162 non-null  object
 5   relationship    30162 non-null  object
 6   race            30162 non-null  object
 7   sex             30162 non-null  object
 8   capital_gain    30162 non-null  int64 
 9   capital_loss    30162 non-null  int64 
 10  hours_per_week  30162 non-null  int64 
 11  native_country  30162 non-null  object
dtypes: int64(5), object(7)
memory usage: 2.8+ MB


In [5]:
X_train.describe()

,age,education_num,capital_gain,capital_loss,hours_per_week
count,30162.000000,30162.000000,30162.000000,30162.000000,30162.000000
mean,38.437902,10.121312,1092.007858,88.372489,40.931238
std,13.134665,2.549995,7406.346497,404.298370,11.979984
min,17.000000,1.000000,0.000000,0.000000,1.000000
25%,28.000000,9.000000,0.000000,0.000000,40.000000
50%,37.000000,10.000000,0.000000,0.000000,40.000000
75%,47.000000,13.000000,0.000000,0.000000,45.000000
max,90.000000,16.000000,99999.000000,4356.000000,99.000000


In [21]:
print(X_train['capital_gain'].quantile([q/20 for q in range(15, 21)]))

0.75        0.0
0.80        0.0
0.85        0.0
0.90        0.0
0.95     5013.0
1.00    99999.0
Name: capital_gain, dtype: float64


In [23]:
print(X_train['capital_loss'].quantile([q/20 for q in range(15, 21)]))

0.75       0.0
0.80       0.0
0.85       0.0
0.90       0.0
0.95       0.0
1.00    4356.0
Name: capital_loss, dtype: float64


### capital_gain, capital_loss 둘 다 우측으로 크게 치우쳐 있다.

In [24]:
# 파생변수 만들기
X_train['capital_gain_yn'] = np.where(X_train['capital_gain']>0, 1, 0) # 0보다 크면 1, 작으면 0
X_train['capital_loss_yn'] = np.where(X_train['capital_loss']>0, 1, 0)

X_test['capital_gain_yn'] = np.where(X_test['capital_gain']>0, 1, 0)
X_test['capital_loss_yn'] = np.where(X_test['capital_loss']>0, 1, 0)

In [26]:
# 변수 종류 별로 나누기 
COL_DEL = []
COL_NUM = ['age', 'education_num', 'hours_per_week', 'capital_gain', 'capital_loss']
COL_CAT = ['workclass', 'marital_status', 'occupation', 'relationship', 'race', 'sex', 'native_country',
          'capital_gain_yn', 'capital_loss_yn']
COL_Y = ['target']

X_train = X_train.drop(COL_DEL, axis=1)
X_test = X_test.drop(COL_DEL, axis=1)

In [27]:
# 수치형 변수에 대해 종속변수 값 별로 각 독립변수 분포 확인
train_df = pd.concat([X_train, y_train], axis=1)

for _col in COL_NUM:
    print('-'*80)
    print(_col)
    print(train_df.groupby(COL_Y)[_col].describe(), end='\n\n')

--------------------------------------------------------------------------------
age
          count      mean        std   min   25%   50%   75%   max
target                                                            
0       22654.0  36.60806  13.464631  17.0  26.0  34.0  45.0  90.0
1        7508.0  43.95911  10.269633  19.0  36.0  43.0  51.0  90.0

--------------------------------------------------------------------------------
education_num
          count       mean       std  min   25%   50%   75%   max
target                                                           
0       22654.0   9.629116  2.413596  1.0   9.0   9.0  10.0  16.0
1        7508.0  11.606420  2.368423  2.0  10.0  12.0  13.0  16.0

--------------------------------------------------------------------------------
hours_per_week
          count       mean        std  min   25%   50%   75%   max
target                                                            
0       22654.0  39.348592  11.950774  1.0  38.0  40.0  

### 해석
- target이 1일 때 mean 값이 0일 때보다 크다
- capital_gain, capital_loss는 종속변수 값에 따른 평균 값 차이가 더 크다. 수치형 변수 모두 종속변수 예측에 사용할 만 하다


In [28]:
# 각 범주형 변수 별로 범주값 별 종속변수 평균 확인
# 범주값에 따라 종속변수가 1인 비율이 차이가 있는지 간단히 확인
for _col in COL_CAT:
    print(train_df.groupby(_col, as_index=False)[COL_Y].mean().sort_values(by=COL_Y, ascending=False), end='\n\n')

          workclass    target
3      Self-emp-inc  0.558659
0       Federal-gov  0.387063
1         Local-gov  0.294630
4  Self-emp-not-inc  0.285714
5         State-gov  0.268960
2           Private  0.218792
6       Without-pay  0.000000

          marital_status    target
1      Married-AF-spouse  0.476190
2     Married-civ-spouse  0.454959
0               Divorced  0.107262
6                Widowed  0.096735
3  Married-spouse-absent  0.083784
5              Separated  0.070288
4          Never-married  0.048324

           occupation    target
3     Exec-managerial  0.485220
9      Prof-specialty  0.448489
10    Protective-serv  0.326087
12       Tech-support  0.304825
11              Sales  0.270647
2        Craft-repair  0.225310
13   Transport-moving  0.202926
0        Adm-clerical  0.133835
6   Machine-op-inspct  0.124619
4     Farming-fishing  0.116279
1        Armed-Forces  0.111111
5   Handlers-cleaners  0.061481
7       Other-service  0.041096
8     Priv-house-serv  0.00699

### 해석
- native country : 41개의 고유값 -> 카디널리티 높은 변수는 모델 성능에 악영향 -> 변수 제거하거나 일부 범주 합쳐서 낮추기 가능 or target encoding
- 우리는 일단 데이터 양이 적지 않고, 카디널리티가 엄청 높은게 아니니까 우려하지 않아도 될 듯

In [34]:
# 범주형 변수들에 대해 label encoding
from sklearn.preprocessing import LabelEncoder

X = pd.concat([X_train, X_test])

for _col in COL_CAT:
    le = LabelEncoder()
    le.fit(X_train[_col])
    X_train[_col] = le.transform(X_train[_col])
    X_test[_col] = le.transform(X_test[_col])

* label encoding을 한 데이터는 선형 모델에 적합하지 않음 = 인코딩 한 변수 값이 의미를 가지는게 아니기 때문
* 따라서, 이어지는 데이터 모형 구축 시에는 비선형 모델인 랜덤포레스트, XGBoost 알고리즘 사용하기
### 4. 데이터 모형 구축

In [43]:
from sklearn.model_selection import train_test_split

X_tr, X_val, y_tr, y_val = train_test_split(X_train, y_train, test_size=0.3, stratify=y_train)

In [44]:
from sklearn.preprocessing import StandardScaler

# scaler의 기준은 train dataset, validation, test set에 적용 동일하게
scaler = StandardScaler()
X_tr[COL_NUM] = scaler.fit_transform(X_tr[COL_NUM])
X_val[COL_NUM] = scaler.transform(X_val[COL_NUM])
X_test[COL_NUM] = scaler.transform(X_test[COL_NUM])

In [46]:
# 모델링
from sklearn.ensemble import RandomForestClassifier

model_rf = RandomForestClassifier()
model_rf.fit(X_tr, y_tr.values.ravel())

RandomForestClassifier()

In [47]:
from xgboost import XGBClassifier

model_xgb1 = XGBClassifier()
model_xgb1.fit(X_tr, y_tr.values.ravel())

[15:42:32] WARNING: /opt/concourse/worker/volumes/live/7a2b9f41-3287-451b-6691-43e9a6c0910f/volume/xgboost-split_1619728204606/work/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=8, num_parallel_tree=1, random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [48]:
# XGB 학습 시 eval_set을 사용하여 충분히 학습 시키면서 과적합 방지 가능
model_xgb2 = XGBClassifier(n_estimators = 1000, learning_rate=0.1, max_depth=10)
model_xgb2.fit(X_tr, y_tr.values.ravel(), early_stopping_rounds = 50, eval_metric='auc', eval_set = [(X_val, y_val)], verbose=10)

[0]	validation_0-auc:0.90208
[10]	validation_0-auc:0.91094
[20]	validation_0-auc:0.91674
[30]	validation_0-auc:0.91980
[40]	validation_0-auc:0.92237
[50]	validation_0-auc:0.92380
[60]	validation_0-auc:0.92431
[70]	validation_0-auc:0.92478
[80]	validation_0-auc:0.92514
[90]	validation_0-auc:0.92532
[100]	validation_0-auc:0.92542
[110]	validation_0-auc:0.92552
[120]	validation_0-auc:0.92536
[130]	validation_0-auc:0.92542
[140]	validation_0-auc:0.92544
[150]	validation_0-auc:0.92534
[160]	validation_0-auc:0.92531
[162]	validation_0-auc:0.92518


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.1, max_delta_step=0, max_depth=10,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=1000, n_jobs=8, num_parallel_tree=1, random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [51]:
# 초도 모델의 성능을 확인하기 위해 검증 데이터 이용해서 ROC-AUC 값 구하기 
from sklearn.metrics import roc_auc_score

y_pred_rf = model_rf.predict_proba(X_val)
y_pred_xgb1 = model_xgb1.predict_proba(X_val)

score_rf = roc_auc_score(y_val, y_pred_rf[:, 1])
score_xgb1 = roc_auc_score(y_val, y_pred_xgb1[:, 1])

print(score_rf)
print(score_xgb1)

0.8982575376086671
0.9262729795900447


In [56]:
# capital_gain_yn, capital_loss_yn이 예측이 도움이 됐는지 확인
pd.DataFrame({'feature':X_tr.columns, 'fi_rf':model_rf.feature_importances_, 'fi_xgb':model_xgb1.feature_importances_})

,feature,fi_rf,fi_xgb
0,age,0.216098,0.032499
1,workclass,0.048425,0.021587
2,education_num,0.138969,0.098923
3,marital_status,0.070635,0.046245
4,occupation,0.088244,0.032052
5,relationship,0.113904,0.447962
6,race,0.018586,0.017377
7,sex,0.011043,0.033890
8,capital_gain,0.098650,0.162117
9,capital_loss,0.032707,0.065699


### 해석
- capital_gain_yn, capital_loss_yn은 변수 중요도가 굉장히 낮다.
- capital_gain, capital_loss가 오히려 더 중요도가 높다.
- 그러므로, capital_gain_yn, capital_loss_yn은 삭제

In [57]:
COL_DEL = ['capital_gain_yn', 'capital_loss_yn']

X_tr = X_tr.drop(COL_DEL, axis=1)
X_val = X_val.drop(COL_DEL, axis=1)
X_test = X_test.drop(COL_DEL, axis=1)

### RandomForest Hyper Parameter Tuning

In [59]:
from sklearn.model_selection import GridSearchCV

grid_params = {
    'n_estimators' : [50, 100, 200],
    'max_depth' : [5, 10, 15],
    'min_samples_split' : [2, 5, 10],
    'min_samples_leaf' : [1, 2, 4]
}

rf_cv = GridSearchCV(estimator=model_rf, param_grid=grid_params, cv=5)
rf_cv.fit(X_train, y_train.values.ravel())

GridSearchCV(cv=5, estimator=RandomForestClassifier(),
             param_grid={'max_depth': [5, 10, 15],
                         'min_samples_leaf': [1, 2, 4],
                         'min_samples_split': [2, 5, 10],
                         'n_estimators': [50, 100, 200]})

## Tip
- 시험 응시 시 전체 코드 실행 시간은 1분 제한조건 
- 위 코드는 제대로 제출되지 않을 수 있음
- 너무 오래 걸린다면 하이퍼파라미터는 2~3개씩 나눠서 튜닝 수행할 수도 있음

In [60]:
print(pd.DataFrame(rf_cv.cv_results_).head())

   mean_fit_time  std_fit_time  mean_score_time  std_score_time  \
0       0.247149      0.013084         0.016269        0.000064   
1       0.480305      0.002304         0.031155        0.000224   
2       0.969255      0.019273         0.061806        0.002122   
3       0.246363      0.007617         0.016627        0.000227   
4       0.486308      0.012491         0.031476        0.000524   

  param_max_depth param_min_samples_leaf param_min_samples_split  \
0               5                      1                       2   
1               5                      1                       2   
2               5                      1                       2   
3               5                      1                       5   
4               5                      1                       5   

  param_n_estimators                                             params  \
0                 50  {'max_depth': 5, 'min_samples_leaf': 1, 'min_s...   
1                100  {'max_depth': 5,

In [61]:
print(rf_cv.best_params_)

{'max_depth': 15, 'min_samples_leaf': 1, 'min_samples_split': 10, 'n_estimators': 100}


In [63]:
# 튜닝한 하이퍼파라미터로 새로운 모델 생성
model_rf2 = RandomForestClassifier(n_estimators = 50,
                        max_depth=15,
                        min_samples_leaf = 1,
                        min_samples_split = 5)
model_rf2.fit(X_tr, y_tr.values.ravel())

y_pred_rf2 = model_rf2.predict_proba(X_val)
score_rf2 = roc_auc_score(y_val, y_pred_rf2[:, 1])
print(score_rf2)

0.9164367912810766


In [64]:
# 좀 짧은 수행시간으로 ... 
grid_params = {
    'max_depth' : [3, 5, 7, 10],
    'min_child_weight' : [1, 2],
    'colsample_bytree' : [0.6, 0.8],
    'subsample' : [0.6, 0.8]
}
xgb_cv = GridSearchCV(estimator=model_xgb1, param_grid=grid_params, cv=5)
xgb_cv.fit(X_tr, y_tr.values.ravel())

[16:03:58] WARNING: /opt/concourse/worker/volumes/live/7a2b9f41-3287-451b-6691-43e9a6c0910f/volume/xgboost-split_1619728204606/work/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[16:03:59] WARNING: /opt/concourse/worker/volumes/live/7a2b9f41-3287-451b-6691-43e9a6c0910f/volume/xgboost-split_1619728204606/work/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[16:03:59] WARNING: /opt/concourse/worker/volumes/live/7a2b9f41-3287-451b-6691-43e9a6c0910f/volume/xgboost-split_1619728204606/work/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' t

[16:04:05] WARNING: /opt/concourse/worker/volumes/live/7a2b9f41-3287-451b-6691-43e9a6c0910f/volume/xgboost-split_1619728204606/work/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[16:04:06] WARNING: /opt/concourse/worker/volumes/live/7a2b9f41-3287-451b-6691-43e9a6c0910f/volume/xgboost-split_1619728204606/work/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[16:04:06] WARNING: /opt/concourse/worker/volumes/live/7a2b9f41-3287-451b-6691-43e9a6c0910f/volume/xgboost-split_1619728204606/work/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' t

[16:04:16] WARNING: /opt/concourse/worker/volumes/live/7a2b9f41-3287-451b-6691-43e9a6c0910f/volume/xgboost-split_1619728204606/work/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[16:04:16] WARNING: /opt/concourse/worker/volumes/live/7a2b9f41-3287-451b-6691-43e9a6c0910f/volume/xgboost-split_1619728204606/work/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[16:04:17] WARNING: /opt/concourse/worker/volumes/live/7a2b9f41-3287-451b-6691-43e9a6c0910f/volume/xgboost-split_1619728204606/work/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' t

[16:04:30] WARNING: /opt/concourse/worker/volumes/live/7a2b9f41-3287-451b-6691-43e9a6c0910f/volume/xgboost-split_1619728204606/work/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[16:04:30] WARNING: /opt/concourse/worker/volumes/live/7a2b9f41-3287-451b-6691-43e9a6c0910f/volume/xgboost-split_1619728204606/work/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[16:04:31] WARNING: /opt/concourse/worker/volumes/live/7a2b9f41-3287-451b-6691-43e9a6c0910f/volume/xgboost-split_1619728204606/work/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' t

[16:04:41] WARNING: /opt/concourse/worker/volumes/live/7a2b9f41-3287-451b-6691-43e9a6c0910f/volume/xgboost-split_1619728204606/work/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[16:04:41] WARNING: /opt/concourse/worker/volumes/live/7a2b9f41-3287-451b-6691-43e9a6c0910f/volume/xgboost-split_1619728204606/work/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[16:04:41] WARNING: /opt/concourse/worker/volumes/live/7a2b9f41-3287-451b-6691-43e9a6c0910f/volume/xgboost-split_1619728204606/work/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' t

[16:04:49] WARNING: /opt/concourse/worker/volumes/live/7a2b9f41-3287-451b-6691-43e9a6c0910f/volume/xgboost-split_1619728204606/work/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[16:04:50] WARNING: /opt/concourse/worker/volumes/live/7a2b9f41-3287-451b-6691-43e9a6c0910f/volume/xgboost-split_1619728204606/work/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[16:04:50] WARNING: /opt/concourse/worker/volumes/live/7a2b9f41-3287-451b-6691-43e9a6c0910f/volume/xgboost-split_1619728204606/work/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' t

[16:05:02] WARNING: /opt/concourse/worker/volumes/live/7a2b9f41-3287-451b-6691-43e9a6c0910f/volume/xgboost-split_1619728204606/work/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[16:05:02] WARNING: /opt/concourse/worker/volumes/live/7a2b9f41-3287-451b-6691-43e9a6c0910f/volume/xgboost-split_1619728204606/work/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[16:05:03] WARNING: /opt/concourse/worker/volumes/live/7a2b9f41-3287-451b-6691-43e9a6c0910f/volume/xgboost-split_1619728204606/work/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' t

GridSearchCV(cv=5,
             estimator=XGBClassifier(base_score=0.5, booster='gbtree',
                                     colsample_bylevel=1, colsample_bynode=1,
                                     colsample_bytree=1, gamma=0, gpu_id=-1,
                                     importance_type='gain',
                                     interaction_constraints='',
                                     learning_rate=0.300000012,
                                     max_delta_step=0, max_depth=6,
                                     min_child_weight=1, missing=nan,
                                     monotone_constraints='()',
                                     n_estimators=100, n_jobs=8,
                                     num_parallel_tree=1, random_state=0,
                                     reg_alpha=0, reg_lambda=1,
                                     scale_pos_weight=1, subsample=1,
                                     tree_method='exact', validate_parameters=1,
         

In [65]:
print(xgb_cv.best_estimator_)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.6, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=5,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=8, num_parallel_tree=1, random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=0.8,
              tree_method='exact', validate_parameters=1, verbosity=None)


In [66]:
params = {'colsample_bytree': 0.6,
         'max_depth' : 7, 
         'min_child_weight' : 1,
         'subsample' : 0.8}

model_xgb3 = XGBClassifier(n_estimators=1000, learning_rate = 0.05)
model_xgb3.set_params(**params)

model_xgb3.fit(X_tr, y_tr, early_stopping_rounds=50, eval_metric='auc', eval_set=[(X_val, y_val)], verbose=10)

print(model_xgb3.best_score)

[0]	validation_0-auc:0.85773
[10]	validation_0-auc:0.90636
[20]	validation_0-auc:0.91224
[30]	validation_0-auc:0.91660
[40]	validation_0-auc:0.91795
[50]	validation_0-auc:0.91913
[60]	validation_0-auc:0.92072
[70]	validation_0-auc:0.92190
[80]	validation_0-auc:0.92314
[90]	validation_0-auc:0.92371
[100]	validation_0-auc:0.92456
[110]	validation_0-auc:0.92543
[120]	validation_0-auc:0.92565
[130]	validation_0-auc:0.92623
[140]	validation_0-auc:0.92654
[150]	validation_0-auc:0.92682
[160]	validation_0-auc:0.92724
[170]	validation_0-auc:0.92753
[180]	validation_0-auc:0.92759
[190]	validation_0-auc:0.92788
[200]	validation_0-auc:0.92806
[210]	validation_0-auc:0.92816
[220]	validation_0-auc:0.92816
[230]	validation_0-auc:0.92820
[240]	validation_0-auc:0.92831
[250]	validation_0-auc:0.92832
[260]	validation_0-auc:0.92826
[270]	validation_0-auc:0.92836
[280]	validation_0-auc:0.92831
[290]	validation_0-auc:0.92818
[300]	validation_0-auc:0.92819
[310]	validation_0-auc:0.92821
[320]	validation_0-

In [67]:
# 튜닝한 xgb 모델이 가장 좋은 성능을 보임 
# 요구하는 제출물 만들기 (1에 해당하는 pred 값 추출)
pred = model_xgb3.predict_proba(X_test)[:, 1]
pd.DataFrame({'index':X_test.index, 'target':pred}).to_csv('005000000.csv', index=False)